In [1]:
import gdspy
import pandas as pd
import numpy as np
import sys
import math
import pattern_writer as pw

In [2]:
baseUnit = 1 #layout scale(microns)
m=baseUnit*1e6 
nm = m*1E-9

In [3]:
'''def layout_placement(basisX, basisY, numberX, numberY, generator_function, placement_function):
    Xs=[]
    Ys=[]
    lastX=0
    lastY=0
    while len(Xs)< numberX:
        Xs.append(lastX)
        lastX+=basisX
    while len(Ys)< numberY:
        Ys.append(lastY)
        lastY+=basisY'''

'def layout_placement(basisX, basisY, numberX, numberY, generator_function, placement_function):\n    Xs=[]\n    Ys=[]\n    lastX=0\n    lastY=0\n    while len(Xs)< numberX:\n        Xs.append(lastX)\n        lastX+=basisX\n    while len(Ys)< numberY:\n        Ys.append(lastY)\n        lastY+=basisY'

In [4]:
def layout_placement(basisX, basisY, numberX, numberY, generator_function, placement_function):
    array=[]
    lastX=0
    for nX in range(numberX):
        lastY=0
        for nY in range(numberY):
            geometry_cell= generator_function(placement_function(lastX, lastY))
            array.append(gdspy.CellReference(geometry_cell, (lastX, lastY), magnification=1))
            #pw.array_singleReference(lastX, lastY, geometry_cell)
            lastY+=basisY
        lastX+=basisX
    return array

In [5]:
#geometry1 = gdspy.Cell("CIRCLE")
#geometry1.add(gdspy.Round((0, 0), 93*nm, tolerance=0.001))
#geometry2 = gdspy.Cell("OVAL")
#geometry2.add(gdspy.Round((0, 0), [93*nm, 80*nm], tolerance=1e-4))
#def p_f(x,y):
 #   return np.sqrt(x**2 + y**2)
#def g_f(distance):
 #   if distance < 200*nm:
        #geometry1 = gdspy.Cell("CIRCLE")
        #geometry1.add(gdspy.Round((0, 0), 93*nm, tolerance=0.001))
  #      return geometry1
   # else:
        #geometry2 = gdspy.Cell("OVAL")
        #geometry2.add(gdspy.Round((0, 0), [93*nm, 80*nm], tolerance=1e-4))
    #    return geometry2
#y = layout_placement(300*nm,100*nm,2,3,g_f, p_f)

In [6]:
#y

In [7]:
def layout_placement2(basisX, basisY, numberX, numberY, generator_function, placement_function):
    array=[]
    array_X=[]
    array_Y=[]
    lastX=0
    for nX in range(numberX):
        array_X.append(lastX)
        lastY=0
        for nY in range(numberY):
            if len(array_Y)<numberY:
                array_Y.append(lastY)
            geometry_cell= generator_function(placement_function(lastX, lastY))
            array.append(gdspy.CellReference(geometry_cell, (lastX, lastY), magnification=1))
            #pw.array_singleReference(lastX, lastY, geometry_cell)
            lastY+=(2*basisY)
        lastX+=(2*basisX)
    array2_X= np.add(array_X, basisX).tolist()
    array2_Y= np.add(array_Y, basisY).tolist()
    #print(array2_Y)
    for n2X in array2_X:
        for n2Y in array2_Y:
            geometry_cell= generator_function(placement_function(n2X, n2Y))
            array.append(gdspy.CellReference(geometry_cell, (n2X, n2Y), magnification=1))
    return array

In [8]:
geometry1 = gdspy.Cell("CIRCLE")
geometry1.add(gdspy.Round((0, 0), 93*nm, tolerance=0.001))
geometry2 = gdspy.Cell("OVAL")
geometry2.add(gdspy.Round((0, 0), [93*nm, 80*nm], tolerance=1e-4))
def p_f(x,y):
    return np.sqrt(x**2 + y**2)
def g_f(distance):
    if distance < 500*nm:
        #geometry1 = gdspy.Cell("CIRCLE")
        #geometry1.add(gdspy.Round((0, 0), 93*nm, tolerance=0.001))
        return geometry1
    else:
        #geometry2 = gdspy.Cell("OVAL")
        #geometry2.add(gdspy.Round((0, 0), [93*nm, 80*nm], tolerance=1e-4))
        return geometry2
y = layout_placement2(300*nm,100*nm,5,8,g_f, p_f)

In [9]:
y

[CellReference("CIRCLE", (0, 0), None, 1, False),
 CellReference("CIRCLE", (0, 0.2), None, 1, False),
 CellReference("CIRCLE", (0, 0.4), None, 1, False),
 CellReference("OVAL", (0, 0.6000000000000001), None, 1, False),
 CellReference("OVAL", (0, 0.8), None, 1, False),
 CellReference("OVAL", (0, 1.0), None, 1, False),
 CellReference("OVAL", (0, 1.2), None, 1, False),
 CellReference("OVAL", (0, 1.4), None, 1, False),
 CellReference("OVAL", (0.6, 0), None, 1, False),
 CellReference("OVAL", (0.6, 0.2), None, 1, False),
 CellReference("OVAL", (0.6, 0.4), None, 1, False),
 CellReference("OVAL", (0.6, 0.6000000000000001), None, 1, False),
 CellReference("OVAL", (0.6, 0.8), None, 1, False),
 CellReference("OVAL", (0.6, 1.0), None, 1, False),
 CellReference("OVAL", (0.6, 1.2), None, 1, False),
 CellReference("OVAL", (0.6, 1.4), None, 1, False),
 CellReference("OVAL", (1.2, 0), None, 1, False),
 CellReference("OVAL", (1.2, 0.2), None, 1, False),
 CellReference("OVAL", (1.2, 0.4), None, 1, False)

In [10]:
gdspy.current_library = gdspy.GdsLibrary()
Lens = gdspy.Cell("LENS")
Lens.add(y)
gdspy.current_library.add(Lens)
gdspy.write_gds('Testing12.gds')